<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [ ]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

In some cases it is desirable to be able to perform operations (e.g. image cutouts) on the server side to avoid transferring large amounts of data. This is why it is useful to use this cutout service. See https://www.ivoa.net/documents/DataLink/20211115/WD-DataLink-1.1-20211115.htmlhttps://www.ivoa.net/documents/DataLink/20211115/WD-DataLink-1.1-20211115.html where it says Access Data Services. This is a useful link: https://dmtn-238.lsst.io/https://dmtn-238.lsst.io/



### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [ ]:
# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

# Import general python packages
import time
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests
import warnings
import matplotlib.pyplot as plt
import os
plt.style.use('tableau-colorblind10')

# Science Pipelines imports
from lsst.daf.butler import Butler, DatasetType, CollectionType
import lsst.geom as geom
import lsst.resources
import lsst.afw.image as afwImage
from lsst.afw.image import Image, ImageF
from lsst.afw.image import Exposure, ExposureF

# Plotting with MPL
import matplotlib.pyplot as plt

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token

# PyVO packages
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

# Astropy
from astropy import units as u
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.visualization import  ZScaleInterval, AsinhStretch
from astropy.wcs import WCS       
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize
from astropy.visualization.stretch import SinhStretch, LinearStretch, SqrtStretch

# Holoviz for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

# stuff for Jeff's warping
import glob
from lsst.pipe.tasks.registerImage import RegisterConfig, RegisterTask
from PIL import Image


### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

In [ ]:
# Set up some plotting defaults
params = {'axes.labelsize': 18,
          'font.size': 18,
          'legend.fontsize': 12,
          'xtick.major.width': 2,
          'xtick.minor.width': 1,
          'xtick.major.size': 10,
          'xtick.minor.size': 4,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 2,
          'axes.linewidth': 2,
          'axes.labelweight': 2,
          'axes.titleweight': 2,
          'ytick.major.width': 2,
          'ytick.minor.width': 1,
          'ytick.major.size': 10,
          'ytick.minor.size': 4,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [6, 6],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

# Set afw display backend to matplotlib
afwDisplay.setDefaultBackend('matplotlib')

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)


# make a temporary working directory
# specify the path for the directory – make sure to surround it with quotation marks

homeDir = os.getenv("HOME")
tempdir = homeDir+'/temp'
if not os.path.exists(tempdir):
    os.makedirs(tempdir)


In [ ]:
def plotImage(exposure: ExposureF, img_opt: dict = None):
    """Plot and image using matplotlib
   
   Parameters
    ----------
    image : `Exposure`
        the image to plot
        
    opts : ``
   
   Returns
    -------
    title : `str` (only if result is not `None`)
        Plot title from string
    """
    
    fig, ax = plt.subplots()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(exposure.image)
    plt.show()


In [ ]:
def make_image_cutout(tap_service, ra, dec, cutout_size=0.01, 
                      imtype=None, dataId=None, filename=None): #visit=None, detector=None, filename=None):
    """Make a cutout using the cutout tool
   
   Parameters
    ----------
    # cutout_size is in degrees
    # if imtype None, assumes deepCoadd. dataId must correspond to that for provided imtype
    # TBD supplying band as an option for deepCoadd
    # get difference images too (i.e. new, reference/template, and difference image). might want to 
    # (not in this function but down the road in our package of utilities to have triplet)
    # tutorial should demonstrate volume of cutouts (i.e. what i am already doing)
   Returns
    -------
    
    """

    spherePoint = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
    
    if imtype == 'calexp':
        
        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_visit, lsst_detector, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.calexp' " + \
            "AND lsst_visit = " + str(dataId["visit"]) + " " + \
            "AND lsst_detector = " + str(dataId["detector"])
        results = tap_service.search(query) # THIS IS TAP SERVICE

    else:
        # Find the tract and patch that contain this point
        # this is how its done using butler:
        tract = dataId["tract"]
        patch = dataId["patch"]
        
        # add optional default band if it is not contained in the dataId
        band = dataId["band"]
        print(band)

        query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
            "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
            "AND obs_collection = 'LSST.DP02' " + \
            "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
            "AND lsst_tract = " + str(tract) + " " + \
            "AND lsst_patch = " + str(patch) + " " + \
            "AND lsst_band = " + "'" + str(band) + "' "
        results = tap_service.search(query) 

    #Get datalink
    dataLinkUrl = results[0].getdataurl()
    f"Datalink link service url: {dataLinkUrl}"
    auth_session = service._session
    dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
    f"{dl_results.status}"

    # from_resource: creates a instance from a number of records and a Datalink Resource.
    sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                                 session=auth_session)

    sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
                 spherePoint.getDec().asDegrees()*u.deg, 
                 cutout_size* u.deg)

    #sodaCutout = os.path.join(os.getenv('HOME'), 'DATA/soda-cutout.fits')
    if filename:
        #sodaCutout = 'cutouts/'+filename
        sodaCutout = os.path.join(os.getenv('HOME'), 'temp/'+filename)
    else:
        #sodaCutout = 'cutouts/soda-cutout.fits'
        sodaCutout = os.path.join(os.getenv('HOME'), 'temp/soda-cutout.fits')

    with open(sodaCutout, 'bw') as f:
        f.write(sq.execute_stream().read())
        
    #sodaCutout is just the filename...
    return sodaCutout


In [ ]:
# some functions from Jeff and Ryan to facilitate animations
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

def get_minmax_xy(img, cutout_size):

    cutout_size = int(cutout_size)
    
    height = img.height
    width = img.width

    ceny = (height - 1) / 2
    cenx = (width - 1) / 2

    minx = int(cenx - ((cutout_size - 1) / 2))
    maxx = int(cenx + ((cutout_size - 1) / 2))
    miny = int(ceny - ((cutout_size - 1) / 2))
    maxy = int(ceny + ((cutout_size - 1) / 2))

    return {'minx':minx, 'maxx':maxx, 'miny':miny, 'maxy':maxy}

def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))]
    frame_one = frames[0]
    frame_one.save("animation.gif", format="GIF", append_images=frames,
               save_all=True, duration=500, loop=1)


## 2. Generating an image cutout using the cutout tool

First, do example with a deepCoadd

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry
#butler = Butler(config, collections=collection)


In [ ]:
service = get_tap_service()

In [ ]:
# galaxy cluster:
#ra = 55.7467 
#dec = -32.2862

# vairable star from tutorial 07b
#ra_known_rrl  = 62.1479031
#dec_known_rrl  = -35.799138

# random DIAobject from a testing code (this ra/dec don't work for some reason
ra = 72.5383603
dec = -44.4248533
diaObjectId = 1253478440036730088

# Define a spatial point
spherePoint = lsst.geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)

#55.75067151347799 -32.27781914678513

# Find the tract and patch that contain this point
skymap = butler.get('skyMap') # skymap is used for coadd projection
tract = skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)

# Now create a dataId from the tract, patch and filter information
dataId = {'band':'i', 'tract':tract.tract_id, 'patch': patch.getSequentialIndex()}

# And use the dataid to get the coadd
deepCoadd = butler.get('deepCoadd_calexp', dataId=dataId)

assert type(deepCoadd) == lsst.afw.image.ExposureF
f"Tract: {tract.tract_id}, Patch: {patch.getSequentialIndex()}"


# Select all images in a region
#query = """SELECT * FROM ivoa.ObsCore 
#WHERE dataproduct_type = 'image'
#AND obs_collection = 'LSST.DP02' 
#AND dataproduct_subtype = 'lsst.deepCoadd_calexp'
#AND CONTAINS(POINT('ICRS', 55.74673760481304, -32.286155241413624), s_region)=1 
#"""
#results = service.search(query).to_table()
#results[1]
#results.to_table().show_in_notebook()

In [ ]:
t = tract.tract_id
p = patch.getSequentialIndex()

print(tract.tract_id, patch.getSequentialIndex())

query = "SELECT access_format, access_url, dataproduct_subtype, lsst_patch, lsst_tract, " + \
    "lsst_band, s_ra, s_dec FROM ivoa.ObsCore WHERE dataproduct_type = 'image' " + \
    "AND obs_collection = 'LSST.DP02' " + \
    "AND dataproduct_subtype = 'lsst.deepCoadd_calexp' " + \
    "AND lsst_tract = " + str(t) + " " + \
    "AND lsst_patch = " + str(p) + " " + \
    "AND lsst_band = 'i' "
    
results = service.search(query)
results.to_table().show_in_notebook()

## Datalink stuff goes here:


In [ ]:
dataLinkUrl = results[0].getdataurl()
f"Datalink link service url: {dataLinkUrl}"
auth_session = service._session
dl_results = DatalinkResults.from_result_url(dataLinkUrl,session=auth_session)
f"{dl_results.status}"

## here is where the cutout tool code starts:


In [ ]:
sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)


In [ ]:
# only 2 shapes supported: circle and polygon. for some reason circle plots as a square (??)

# first do a circle
sphereRadius = 0.001* u.deg

# sq = sodaquery opened earlier
# Defined spatial point was done earlier using spherePoint:
# spherePoint = lsst.geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)

#Example: a circle at (12,34) with radius 0.5:
#CIRCLE=12 34 0.5


sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
             spherePoint.getDec().asDegrees()*u.deg, 
             sphereRadius)


In [ ]:
sq = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)

sq.circle = (spherePoint.getRa().asDegrees()* u.deg,
             spherePoint.getDec().asDegrees()*u.deg, 
             sphereRadius)

# first write the file to disk:
sodaCutout = os.path.join(os.getenv('HOME'), 'temp/soda-cutout.fits')
with open(sodaCutout, 'bw') as f:
    f.write(sq.execute_stream().read())
    
# Display the cutout from the file
plotImage(ExposureF(sodaCutout))


In [ ]:
#Alternatively can use polygon to define a region. Actually, circle is easier because there's less to define
#and it looks like "radius" is also half of the box edge. Here's a demonstration using polygon.

#Example: a polygon from (12,34) to (14,34) to (14,36) to (12,36) and (implicitly) back to (12,34):
#POLYGON=12 34 14 34 14 36 12 36
sq2 = SodaQuery.from_resource(dl_results, dl_results.get_adhocservice_by_id("cutout-sync"), 
                             session=auth_session)
sphereRadius2 = 0.01* u.deg

sq2.polygon = (spherePoint.getRa().asDegrees()* u.deg - sphereRadius2,
              spherePoint.getDec().asDegrees()*u.deg - sphereRadius2,
              spherePoint.getRa().asDegrees()* u.deg - sphereRadius2,
              spherePoint.getDec().asDegrees()*u.deg + sphereRadius2,
              spherePoint.getRa().asDegrees()* u.deg + sphereRadius2,
              spherePoint.getDec().asDegrees()*u.deg + sphereRadius2)
              #spherePoint.getRa().asDegrees()* u.deg + sphereRadius2,
              #spherePoint.getDec().asDegrees()*u.deg - sphereRadius2)


sodaCutout2 = os.path.join(os.getenv('HOME'), 'temp/soda-cutout.fits')
with open(sodaCutout2, 'bw') as f:
    f.write(sq2.execute_stream().read())
    
# Display the cutout
plotImage(ExposureF(sodaCutout2))



### 2.1 Test out a wrapper function "make_image_cutout" defined above, so we can do all the above operations in one go
IT WORKS :-D

In [ ]:
#test2 = make_image_cutout(service, dataid, cutout_size=0.005)
#plotImage(ExposureF(test))
# dataId is for the butler, so don't send?
#print(dataId.values())
#print(dataId)
#tracttest = dataId["patch"]

In [ ]:
# Test our new function wrapper for the cutout service! IT WORKS :-D

imtype = 'deepCoadd' # this is the default, imtype=None will assume deepCoadd
test = make_image_cutout(service, ra, dec, dataId=dataId, imtype=imtype, cutout_size=0.005)
plotImage(ExposureF(test))


## 3. Creating cutouts for validation of Lyman-break selection of high-z galaxies

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 3.1 Section Sub-heading
LBG color selection of z~3 galaxies from Boutsia+14, Steidel+03:

U − G >= 1.20 × (G − R) + 0.96 

G − R >= −0.3 

G − R <= 1.0 

#### 3.1.1 Section Sub-sub-heading

search object catalog (deepCoadd data) for u, r, g_kronFlux and can do this: scisql_nanojanskyToAbMag(g_kronFlux) in the TAP search to grab above some mag limit

In [ ]:
%%time

max_rec = 500
use_center_coords = "62, -37"
ralbg = 62.
declbg = -37.
use_radius = "1.0"

#"FROM dp02_dc2_catalogs.ForcedSource as source "\
#        "JOIN dp02_dc2_catalogs.Object as obj ON source.objectId = obj.objectId "\

# here you will need to also set all U-mags with S/N < 1 to the 1-sig error.
# another idea is to join to MatchesTruth table on matchObjectId and then use id of the matches to join
# on TruthSummary to get redshift

# this appears to select things i want
#query = "SELECT TOP " + str(max_rec) + " " + \
#        "objectId, coord_ra, coord_dec, detect_isPrimary, patch, tract, " + \
#        "u_kronFlux, scisql_nanojanskyToAbMag(u_ap12Flux) as umag, scisql_nanojanskyToAbMag(r_ap12Flux) as rmag, scisql_nanojanskyToAbMag(g_ap12Flux) as gmag " + \
#        "FROM dp02_dc2_catalogs.Object " + \
#        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
#        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
#        "AND detect_isPrimary = 1 " + \
#        "AND g_ap12Flux/g_ap12FluxErr > 10 " + \
#        "AND scisql_nanojanskyToAbMag(g_ap12Flux) < 24 " 
##        "AND g_kronFlux/g_kronFluxErr > 10 " + \

# from Jeff's truth tables tutorial (this does not select things I want)
query = "SELECT TOP " + str(max_rec) + " " + \
        "mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, ts.redshift, "\
        "obj.objectId, obj.coord_ra, obj.coord_dec, obj.detect_isPrimary, obj.patch, obj.tract, " + \
        "obj.u_kronFlux, scisql_nanojanskyToAbMag(obj.u_ap12Flux) as umag, " + \
        "scisql_nanojanskyToAbMag(obj.r_ap12Flux) as rmag, scisql_nanojanskyToAbMag(obj.g_ap12Flux) as gmag " + \
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec),  "\
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND obj.detect_isPrimary = 1 AND ts.redshift > 2.5 " + \
        "AND obj.g_ap12Flux/obj.g_ap12FluxErr > 10 " + \
        "AND scisql_nanojanskyToAbMag(obj.g_ap12Flux) < 24 " 



#"AND scisql_nanojanskyToAbMag(g_kronFlux) - scisql_nanojanskyToAbMag(r_kronFlux) < 1.0 " + \
#"AND scisql_nanojanskyToAbMag(g_kronFlux) - scisql_nanojanskyToAbMag(r_kronFlux) > -0.3 " 

results = service.search(query)
assert len(results) == max_rec
results.to_table()

In [ ]:
#U − G >= 1.20 × (G − R) + 0.96 

# here, set umag to umag of 1-sig error before selecting colors

whlbg = np.where( ((results['umag']-results['gmag']) > (1.20 * (results['gmag'] - results['rmag']) + 0.96 )) & ((results['gmag'] - results['rmag']) < 1.) & ((results['gmag'] - results['rmag']) > 0.3))[0]

color1 = results['gmag']-results['rmag']
color2 = results['umag']-results['gmag']

plt.plot(color1, color2, '.')
plt.plot(color1[whlbg],color2[whlbg],'o')
plt.xlabel('G - R')
plt.ylabel('U - G')
plt.show()
print(results['gmag'][whlbg])
print(results['redshift'][whlbg])

In [ ]:
for i in range(len(whlbg)):
    print(results['redshift'][whlbg][i])
    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'u'}
    testu = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_u_'+str(i)+'.fits')
 
    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'g'}
    testg = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_g_'+str(i)+'.fits')

    dataId_deep = {'patch':results['patch'][whlbg][i], 'tract':results['tract'][whlbg][i], 'band':'r'}
    testr = make_image_cutout(service, results['coord_ra'][whlbg][i], results['coord_dec'][whlbg][i], 
                              cutout_size=0.001,imtype='deepCoadd', dataId=dataId_deep,filename='cutout_r_'+str(i)+'.fits')


    plotImage(ExposureF(testu))
    plotImage(ExposureF(testg))
    plotImage(ExposureF(testr))


## 4. Creating cutouts of changing flux for the diaObject
## 4. Generating a set of calexp cutouts

now do the diasource example here:## 3. Generating a set of calexp cutouts

now do the diasource example here:
_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 4.1 Section Sub-heading



In [ ]:
# find the source by matching the source ID, which gives you the exact source
query = "SELECT TOP " + str(100) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE diaObjectId = " + str(diaObjectId) + " "

# can get filter and MJD from diasource? If so, then maybe you don't have to do the next cell search
# ccdVisitId = visitid+detector where detector starts with 001
# or could join on ccdVisitId (check if this is an optioN).

print(diaObjectId)
visits = service.search(query)
my_tab = visits.to_table()#.to_pandas()
#my_tab['test'] = np.zeros(len(ccdvisit))

# adding cols to pandas tables should be straitforward
# should be able to join the diasource with ccdvisitid tables. join on ccdvisitid 
# adql functionality join will do it. see notebook 02.

In [ ]:
# maybe, instead, use Jeff's join query:
# do we need to sort out diaObjectId vs diaSourceId

ccdquery = "SELECT TOP 10 dia.coord_ra, dia.coord_dec, " + \
                        "dia.diaObjectId,  dia.ccdVisitId, dia.band, " + \
                        "cv.visitId, cv.physical_filter, cv.detector, cv.obsStartMJD, " + \
                         "cv.expMidptMJD " + \
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject as dia " + \
                         "JOIN dp02_dc2_catalogs.CcdVisit as cv ON cv.ccdVisitId = dia.ccdVisitId " + \
                         "WHERE dia.diaObjectId = "+str(diaObjectId)+" AND cv.band = 'i'"
    
diaccdsearch = service.search(ccdquery)
forcedSrc = diaccdsearch.to_table()
forcedSrc
del diaccdsearch

In [ ]:
wh = np.argsort(forcedSrc['obsStartMJD'])
forcedSrc[wh]
#forcedSrc['detector'][wh]

In [ ]:
# This cell plots the cutouts by order of MJD 
#def make_image_cutout(butler, tap_service, ra, dec, cutout_size=0.01, imtype=None, visit=None, detector=None, filename=None):

for i in range(10):
    dataId_calexp = {'visit':forcedSrc['visitId'][wh][i], 'detector':forcedSrc['detector'][wh][i]}
    test = make_image_cutout(service, ra, dec, cutout_size=0.005,imtype='calexp', dataId=dataId_calexp,
                             filename='cutout_'+str(i)+'.fits')
                             #visit=forcedSrc['visitId'][wh][i], detector=forcedSrc['detector'][wh][i])


    plotImage(ExposureF(test))




instead of grayscale, use matplotlib virdis to see if that enhances the transient

In [ ]:
# Jeff's code to generate a gif
if os.path.exists('$HOME/cutouts'):
    print('temp/ directory already exists')
else:
    os.mkdir('$HOME/temp')
    print('Made a temp/ directory')

#if os.path.exists('images'):
#    print('images/ directory already exists')
#else:
#    os.mkdir('images')
#    print('Made an images/ directory')

print(forcedSrc[wh])
#coord_ra	coord_dec	diaObjectId	ccdVisitId	band	visitId	physical_filter	detector	obsStartMJD	expMidptMJD

ii = 0
i = 0
cutout_size_pix = 131

#sys.exit()
#for src in forcedSrc[wh]:#diaSrc:
for i in range(len(wh)):
    visit=forcedSrc['visitId'][wh][i]
    detector=forcedSrc['detector'][wh][i]
    #ccdvisitID = src['ccdVisitId']
    #visit = str(ccdvisitID)[:-3]
    #detector = str(ccdvisitID)[-3:]
    #visit = int(visit)
    #detector = int(detector)
    dataId_calexp = {'visit':visit, 'detector':detector}
    if i == 0:
        # 0.2-arcsec pixels, convert to degrees
        cutout_size = .005#(cutout_size_pix*2) * 0.2/3600.0
        print(cutout_size)
        # Make the first image the reference image that we'll warp all the others to.
        ref_fitsname = 'cutout_ref.fits'
        make_image_cutout(service, ra, dec, cutout_size=cutout_size, imtype='calexp',
                          dataId=dataId_calexp, filename=ref_fitsname)
        img_ref = ExposureF('$HOME/temp/'+ref_fitsname)
        plotImage(img_ref) #this works for plotting
        
        # PICK vmin and vmax based on the reference image for use later
        vmin = -200
        vmax = 300
        
        
    else:
        cutout_size = .005#(cutout_size_pix*1.1) * 0.2/3600.0
        print(cutout_size)
    #ii += 1

        fitsname = 'cutout_'+str(i)+'.fits'
        print(fitsname)
        make_image_cutout(service, ra, dec, cutout_size=cutout_size, imtype='calexp',
                          dataId=dataId_calexp, filename=fitsname)

        img = ExposureF('$HOME/temp/'+fitsname)
        
        plotImage(img) #this works for plotting
    
    
        img_warped = warp_img(img_ref, img, img_ref.getWcs(), img.getWcs())
        
        fig, ax = plt.subplots()
    
        ### CAN AN ERROR BE HERE WITH CUTOUT_SIZE_PIX?
        minmax = get_minmax_xy(img_warped, cutout_size_pix)
    
        # maybe somehting is wrong with the bounding box here. Try removing warped and see if the gif works?
        im_arr = img_warped.image.array[minmax['minx']:minmax['maxx'], minmax['miny']:minmax['maxy']]
        im_arr_bgnorm = im_arr#/np.nanmedian(im_arr)
        
        # Create an ImageNormalize object
        #norm = ImageNormalize(im_arr_bgnorm, interval=ZScaleInterval(),
        #                      stretch=LinearStretch())
        
        plt.imshow(im_arr, origin='lower', cmap='gray', vmin=vmin, vmax=vmax)#norm=norm, 
    
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        #plt.title('MJD: '+str(src['midPointTai'])+'; band: '+str(src['filterName']))
        figname = 'images/cutout_'+str(i)+'.png'
        if os.path.isfile(figname):
            os.remove(figname)
        #plt.show()
        plt.savefig(figname)
        plt.close()
    
sys.exit()


In [ ]:
make_gif('images')
#sys.exit()

notes with melissa:
still TBD:
- demonstrate a failure case
- we dont' yet have a tutorial on galaxy measurements: maybe some sample non-deblended with a larger diversity of shapes and overplot the morph (semi major/minor ellipsoid) 

## Junk code

In [ ]:
# Testing various ways to skip writing to disk here (not working):

#plotImage(ExposureF(sodaCutout))
#plotImage(ExposureF(sq.execute_stream()))

# THIS DOES NOT WORK, and times out after IOPub data rate exceeded:
#x = sq.execute_stream().read()
#print(x)

#image_url = dl_results.getrecord(0).get('access_url')
#plotImage(ExposureF(image_url))
#plotImage(ExposureF(sq))

# This does not work to pass the cutout without saving
#fig = plt.figure()
#display = afwDisplay.Display(frame=fig)
#display.scale('asinh', 'zscale')
#display.mtv(test)
#plt.show()

In [ ]:
# ccdvisittable is a separate catalog that contains all observations and I could use that to extract further info. 
# get detector and visit from ccdvisittable using ccdvisitid. 

ccdqu = "SELECT "  + \
        "ccdVisitId, visitId, physical_filter, band, detector, obsStartMJD " + \
        "FROM dp02_dc2_catalogs.CcdVisit " + \
        "WHERE ccdVisitId = " + str(visits['ccdVisitId'][0]) + " "

# here you have to search on band
# could also return all bands so that you plot all r band and all i band etc sorted by mjd.
# make a colored outline or label the filter int he cutout or something.
# or, chronological order for 1 filter
# user case: want to see if any of the images are bad and so see 

ccdvisit = service.search(ccdqu)
ccdvisit.to_table().to_pandas()
print(ccdvisit)

my_tab['test'] = np.zeros(len(ccdvisit))

In [ ]:
# Write a loop to add these values into one table, so that we can then sort by MJD
# visits = DiaObjs
#print(visits, len(visits))
my_tab['MJD'] = np.zeros(len(visits), dtype='float')
#visits.assign(MJD=0).head()
my_tab['visit'] = np.zeros(len(visits), dtype='float')
my_tab['ccdVisitId_test'] = np.zeros(len(visits), dtype='float')
my_tab['detector'] = np.zeros(len(visits), dtype='float')
my_tab['band'] = np.zeros(len(visits), dtype='float')

# Specify the calexp that we are accessing
#dataId = {'visit': 192350, 'detector': 175, 'band': 'i'}
# Retrieve the data using the `butler` instance and its function `get()`
#calexp = butler.get('calexp', **dataId)


#dont' forget to check Melissa's test NB about imaging in slack
for j in enumerate(visits['ccdVisitId']):
    #print(j)
    ccdqu = "SELECT "  + \
        "ccdVisitId, visitId, physical_filter, band, detector, obsStartMJD " + \
        "FROM dp02_dc2_catalogs.CcdVisit " + \
        "WHERE ccdVisitId = " + str(j[1]) + " " # AND band = 'r' ?
    ccdvisit = service.search(ccdqu).to_table().to_pandas()
    #print(ccdvisit['band'])

    #FSDiaObj['expMidptMJD'][i[0]]=results['expMidptMJD'][0]
    #print(ccdvisit['obsStartMJD'])
    my_tab['MJD'][j[0]] = ccdvisit['obsStartMJD'][0]
    my_tab['visit'][j[0]] = ccdvisit['visitId'][0]
    #my_tab['band'][j[0]] = ccdvisit['band'][0]
    #my_tab['ccdVisitId_test'][j[0]] = ccdvisit['ccdVisitId'][0]
    my_tab['detector'][j[0]] = ccdvisit['detector'][0]
    

    #del ccdvisit



## meeting with Leanne 6/1/23
Look at the sodaquery API to figure out what the format the sq is in. also can type help(sq)
maybe try doing holoviews streaming data instead execute_stream (or execute_raw) or streamz library in holoviews
look at the matplotlib API to see if you can feed it the sq (tried this, doesnt work). for now just write things to file, and proceed with science application. we will revisit plotting directly the image after talkign to DM at the next data reveiw(??) 

## questions from Melissa
 how can we show failure modes (i.e. if you make a cutout near the boundary) or max sizes on the cutout?
 maybe that goes in Leanne's notebooK? we should test what happens

get a SN near a host galaxy so looks interesting
real science use cases: lensed SN (short time delay with multiple images) when seeing is bad the sources may be blended, and if seeing is good they are deblended. people will lok for DIAobjects close togehter because they might be the same physical thing. Source association pipeline split into multiple ones in cases like this. Ryan-this has already happened (2 DIAobj close to same position and some images will have some sources and other images will have other sources identified. Now you can plot all the
 cutouts (2-4" of eachother). DP0.2 doesnt' have lensed transients so just pick nearby DIAobjs as a demonstrated failure mode which is a good use case. ask Ryan about this (ticket open to explore source association for DIAobj) users can get seeing out of the catalogs (check schema in DIAsource catalog, measurement of PSF? ccdvisit table has it for the whole visit

 another failure mode using variable stars close together maybe seeing differences between visits get blended but maybe SN science app is better. 
 
Ask leanne why we need both butler and tap service to do this (can we get either UUID or datalink from one or the other?)

### questions for melissa 4/26:
### - if you want to have a function that uses a butler instance what is the best way to do that?

## From Melissa 4/26:
 we want to avoid having users write things. See if you can do this without
 writing the file to disk. then you can show an example of how to do that
 but in general we want people not to that
 what is function ExposureF? Does F stand for file?



### Answers for Melissa 4/30/23: 
Yes, F stands for file and this tries to open the file written to disk.

To ask Leanne: is this needed? I only see dataurl for the entire image but can the cutout be stored in memory and not written to disk? Also ask: why isn't the circle cutout actually a circle? Why does polygon also let you define a triangle but also returns a rectangle?

To ask Jeff: did he find this slow (is it because reading/writing files?)
Since the visit and detector are required inputs for the calexp case, it might be good to add a try/except block to make sure they’re provided whenever a  calexp cutout is requested.
Since the visit and detector are required inputs for the calexp case, it might be good to add a try/except block to make sure they’re provided whenever a  calexp cutout is requested.


dm_rsp slack channel (soda is a service in RSP).
also could look in NOIRLab datalab because they have some soda tutorials. they might have more info on 
the format of soda returns. also can see who wrote the datalab notebook and see who wrote it and go talk in person.

look at the datalab and see what the use cases they have for sodaqueries or cutouts. 

if doing gif, could do side by side direct image and difference image gifs (run at the same time one is direct image one is difference image). fix the scaling so you can visualize the brihgtness. another application: search object catalog for a bunch of galaxies that had weird shape measurements and look at them all because you're gonna pass them to a citizen science or run galfit on them. 50x50 mosaic of a bunch of weird galaxies. could order them something (color?) and make RGB for differnet cutouts (look in notebook 3a for RGB) 

talk: LSST UK will have an IDAC independent data access center. yous hould know a little about it so that if someone asks me a question about it. Also you should know about LASAIR (the broker i.e. transient alert stream from ZTF for the LSST UK is building) because other countries may have their own proprietary datasets that they want to co-analyze, or they want other analysis tools or osmething. UK has its own datasets that aren't public so we need our own IDAC and our own specific science we want to be able to do. look lsst.org/scientists and look under in-kind programs. email bob mann say i'd like to know more about UK inkind program or other things that are immportant tot he UK that you could share with me so i have a sense of what people are interested in. they may have a document that defines what they want in their IDAC or something. 

lasair-ztf.lsst.ac.uk

From melissa 11/27

hardwire the deleting of cutouts as part of the cleanup
as you develop final steps of notebook start using weekly 47 

switch order so deep coadds first and then the calexp demonstration.
Have notebook make a temp directory 
$HOME/temp and everything should be written thereat the end delete contents and directory itselftutorials notebook repo directory will be read only.
